<a href="https://colab.research.google.com/github/CaretJuice/extract-ga4-event-params-bigquery/blob/main/extract_ga4_event_params_bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Variables

source_project = 'caret-juice-ss'
source_dataset = 'analytics_265010215'
source_table = 'events_intraday_20240205'

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table

Authenticated
The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project=source_project)

events = client.query('''
  select distinct
    event_name
  from {sp}.{sd}.{st} '''.format(sp=source_project, sd=source_dataset, st=source_table) ).result()

event_list=[]
for event in events:
  event_list.append(event[0])
event_list

['first_visit',
 'session_start',
 'form_start',
 'form_submit',
 'user_engagement']

In [ ]:
event_params = {}

for event in event_list:
  event_params[event] = client.query('''
    select
      ep.key
      , count(ep.value.string_value) as string_values
      , count(ep.value.int_value) as int_values
      , count(ep.value.float_value) as float_values
      , count(ep.value.double_value) as double_values
    from {sp}.{sd}.{st}
    cross join
      unnest(event_params) ep
    where event_name = '{en}'
    group by ep.key;
    '''.format(sp=source_project, sd=source_dataset, st=source_table, en=event) ).to_dataframe()



In [ ]:
for event in event_list:
  print (event +'\n')
  print (event_params[event])
  print ('\n')

first_visit

                  key  string_values  int_values  float_values  double_values
0         form_length              0           1             0              0
1       batch_page_id              0           1             0              0
2     ignore_referrer              1           0             0              0
3          page_title              1           0             0              0
4       page_referrer              1           0             0              0
5    form_destination              1           0             0              0
6       page_location              1           0             0              0
7   ga_session_number              0           1             0              0
8     session_engaged              1           0             0              0
9   batch_ordering_id              0           1             0              0
10      ga_session_id              0           1             0              0


session_start

                  key  string_valu